In [21]:
import os
import numpy as np
from nltk.corpus import stopwords
from string import punctuation
from gensim.models import word2vec
from pathlib import Path

In [33]:
folder = Path("C:/Upgrad Projects/NLP-M4")

In [34]:
#os.getcwd()

### Loading the training and test data sets

In [35]:
train_data = folder/"r8-train-all-terms.txt"
test_data = folder/"r8-test-all-terms.txt"

In [36]:
train_data

WindowsPath('C:/Upgrad Projects/NLP-M4/r8-train-all-terms.txt')

In [37]:
X, y = [], []
with open(train_data, "r") as infile:
    for line in infile:
        label, text = line.split("\t")
        X.append(text.split())
        y.append(label)
X, y = np.array(X), np.array(y)
print ("total examples %s" % len(y))

total examples 5485


In [38]:
X_test, y_test = [], []
with open(test_data, "r") as infile:
    for line in infile:
        label, text = line.split("\t")
        X_test.append(text.split())
        y_test.append(label)
X_test, y_test = np.array(X_test), np.array(y_test)
print ("total examples %s" % len(y_test))

total examples 2189


In [39]:
X.shape, X_test.shape

((5485,), (2189,))

In [40]:
np.unique(y, return_counts=True)

(array(['acq', 'crude', 'earn', 'grain', 'interest', 'money-fx', 'ship',
        'trade'], dtype='<U8'),
 array([1596,  253, 2840,   41,  190,  206,  108,  251], dtype=int64))

## Using NB methods

First, reconstructring the string text for our vectorizers to work

In [41]:
X[0:2]

array([list(['champion', 'products', 'ch', 'approves', 'stock', 'split', 'champion', 'products', 'inc', 'said', 'its', 'board', 'of', 'directors', 'approved', 'a', 'two', 'for', 'one', 'stock', 'split', 'of', 'its', 'common', 'shares', 'for', 'shareholders', 'of', 'record', 'as', 'of', 'april', 'the', 'company', 'also', 'said', 'its', 'board', 'voted', 'to', 'recommend', 'to', 'shareholders', 'at', 'the', 'annual', 'meeting', 'april', 'an', 'increase', 'in', 'the', 'authorized', 'capital', 'stock', 'from', 'five', 'mln', 'to', 'mln', 'shares', 'reuter']),
       list(['computer', 'terminal', 'systems', 'cpml', 'completes', 'sale', 'computer', 'terminal', 'systems', 'inc', 'said', 'it', 'has', 'completed', 'the', 'sale', 'of', 'shares', 'of', 'its', 'common', 'stock', 'and', 'warrants', 'to', 'acquire', 'an', 'additional', 'one', 'mln', 'shares', 'to', 'sedio', 'n', 'v', 'of', 'lugano', 'switzerland', 'for', 'dlrs', 'the', 'company', 'said', 'the', 'warrants', 'are', 'exercisable', 'for

In [42]:
X_text = [" ".join(val) for val in X]
print(X_text[0] + "\n")
X_test_text = [" ".join(val) for val in X_test]
print(X_test_text[10])

champion products ch approves stock split champion products inc said its board of directors approved a two for one stock split of its common shares for shareholders of record as of april the company also said its board voted to recommend to shareholders at the annual meeting april an increase in the authorized capital stock from five mln to mln shares reuter

vieille montagne reports loss dividend nil year net loss after exceptional charges mln francs vs profit mln exceptional provisions for closure of viviez electrolysis plant mln francs vs exceptional gain mln sales and services billion francs vs billion proposed net dividend on ordinary shares nil vs francs company s full name is vieille montagne sa vmnb br reuter


#### Using count vectorizer

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

In [44]:
vect = CountVectorizer(stop_words='english',max_features=5000)
vect.fit(X_text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

Applying the vectorizer to our test and train sets

In [45]:
X_train_transformed = vect.transform(X_text)
X_test_transformed =vect.transform(X_test_text)

In [46]:
# printing the vocabulary
list(vect.vocabulary_.items())[:10]

[('champion', 759),
 ('products', 3488),
 ('approves', 264),
 ('stock', 4337),
 ('split', 4273),
 ('said', 3974),
 ('board', 532),
 ('directors', 1327),
 ('approved', 263),
 ('common', 895)]

### Using Bernoulli NB first

In [47]:
from sklearn.metrics import accuracy_score

In [49]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
bnb.fit(X_train_transformed,y)

# predict class
pred_train_ys = bnb.predict(X_train_transformed)
pred_test_ys = bnb.predict(X_test_transformed)

# accuracy
print("Train accuracy: ", accuracy_score(y, pred_train_ys))
print("Test accuracy: ", accuracy_score(y_test, pred_test_ys))

Train accuracy:  0.8736554238833182
Test accuracy:  0.8688899040657835


### Using Multinomial NB
 - We expect this to work very well, giving high performance in accuracy

In [50]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

In [51]:
#fit on training data
mnb.fit(X_train_transformed, y)

# predict class
pred_train_ys = mnb.predict(X_train_transformed)
pred_test_ys = mnb.predict(X_test_transformed)

# accuracy
print("Train accuracy: ", accuracy_score(y, pred_train_ys))
print("Test accuracy: ", accuracy_score(y_test, pred_test_ys))

Train accuracy:  0.968094804010939
Test accuracy:  0.9657377798081316


 - As expected, this performed really well
 - Remember that we used 5000 features!

## Using our word embeddings approach

 We have two options here:
 
1. Use pre-trained word vectors (Glove)

2. Train our own vectors

We'll explore both

### Loading the pre-trained GloVe word vectors

In [16]:
# from gensim.scripts.glove2word2vec import glove2word2vec
# glove_input_file = 'glove.6B.100d.txt'
# word2vec_output_file = 'glove.6B.100d.w2vformat.txt'
# glove2word2vec(glove_input_file, word2vec_output_file)

In [58]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = folder/'glove.6B.200d.txt'
word2vec_output_file = folder/'glove.6B.200d.w2vformat.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

(400000, 200)

In [59]:
from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format(folder/'glove.6B.200d.w2vformat.txt', binary=False)

In [73]:
w1 = 'happy'
glove_model[w1][0:10]

array([ 0.34365 ,  0.34183 ,  0.053328, -0.11949 , -0.26644 , -0.51998 ,
       -0.71326 , -0.12769 ,  0.9092  ,  0.63402 ], dtype=float32)

In [66]:
glove_model.most_similar(w1)

[('glad', 0.7427808046340942),
 ("'m", 0.7389225959777832),
 ('always', 0.7260879874229431),
 ('really', 0.7217552661895752),
 ('everyone', 0.7178582549095154),
 ('pleased', 0.7057917714118958),
 ('wish', 0.7009028196334839),
 ('everybody', 0.698989748954773),
 ('good', 0.6944485902786255),
 ('feel', 0.6898111701011658)]

In [67]:
w2 = 'world'
glove_model[w2][0:10]

array([ 0.46275 ,  0.49719 , -0.47035 , -0.60928 , -0.16782 ,  0.74947 ,
       -0.21528 , -0.15897 ,  0.42159 , -0.059383], dtype=float32)

In [77]:
wv_res1 = np.zeros(glove_model.vector_size)
wv_res1[0:5] 

array([0., 0., 0., 0., 0.])

In [78]:
wv_res1 = wv_res1 + glove_model[w1]
wv_res1[0:5]

array([ 0.34365001,  0.34182999,  0.053328  , -0.11949   , -0.26644   ])

In [79]:
wv_res1 = wv_res1 + glove_model[w2]
wv_res1[0:10]

array([ 0.8064    ,  0.83901998, -0.417022  , -0.72876999, -0.43426001,
        0.22948998, -0.92853999, -0.28666   ,  1.33079001,  0.57463697])

### Sentence vector by averaging word vectors

In [60]:
stop_words = stopwords.words("english")

In [80]:
def sent_vec(sent):
    wv_res = np.zeros(glove_model.vector_size)
    ctr = 1
    for w in sent:
        if w in glove_model:
            ctr += 1
            wv_res += glove_model[w]
    wv_res = wv_res/ctr
    #return (wv_res, ctr)
    return wv_res

In [81]:
train_doc_vecs = []
for doc in X:    
    doc_words = [term for term in doc if term not in stop_words]
    train_doc_vecs.append(sent_vec(doc_words))

In [82]:
test_doc_vecs = []
for doc in X_test:    
    doc_words = [term for term in doc if term not in stop_words]
    test_doc_vecs.append(sent_vec(doc_words))

### Building a predictive model on the averaged word vectors

#### Using a 'simple' logistic regression

In [83]:
from sklearn.linear_model import LogisticRegression

In [84]:
logreg = LogisticRegression(penalty="l1", random_state=42, C = 3.5)
logreg.fit(train_doc_vecs,y)

C:\Users\Naresh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Naresh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=3.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [86]:
pred_train_ys = logreg.predict(train_doc_vecs)
pred_test_ys = logreg.predict(test_doc_vecs)
print("Train accuracy: ", accuracy_score(pred_train_ys, y))
print("Test accuracy: ", accuracy_score(pred_test_ys, y_test))

Train accuracy:  0.9781221513217867
Test accuracy:  0.9680219278209228


### Training our own wordvectors on the data
We'll create a combined text file to train our word vectors - more data is better. Although in this case we would still have just 7.7K instances to learn from.

In [87]:
len(X)

5485

In [88]:
X_comb = np.concatenate([X,X_test])
len(X_comb)

7674

In [89]:
print(X_comb[6000])

['fhlbb', 'changes', 'short', 'term', 'discount', 'note', 'rates', 'the', 'federal', 'home', 'loan', 'bank', 'board', 'adjusted', 'the', 'rates', 'on', 'its', 'short', 'term', 'discount', 'notes', 'as', 'follows', 'maturity', 'new', 'rate', 'old', 'rate', 'maturity', 'days', 'pct', 'pct', 'days', 'days', 'pct', 'pct', 'days', 'days', 'pct', 'pct', 'days', 'days', 'pct', 'pct', 'days', 'days', 'pct', 'pct', 'days', 'reuter']


In [90]:
w2v = word2vec.Word2Vec(X_comb, window=2, min_count=2, sg = 1, size=200)

In [92]:
w2v.wv.most_similar("future", topn=5)

[('near', 0.8028599619865417),
 ('positive', 0.793208658695221),
 ('changes', 0.7894085049629211),
 ('foreseeable', 0.7857812643051147),
 ('internal', 0.7857577204704285)]

#### Sentence vectors by averaging vectors for words

In [104]:
def sent_vec_w2v(sent):
    wv_res = np.zeros(w2v.vector_size)
    ctr = 1
    for w in sent:
        if w in w2v.wv:
            ctr += 1
            wv_res += w2v.wv[w]
    wv_res = wv_res/ctr
    return wv_res

#### Getting the sentence vectors for the test and train sets

In [105]:
train_doc_vecs = []
for doc in X:    
    doc_words = [term for term in doc if term not in stop_words]
    train_doc_vecs.append(sent_vec_w2v(doc_words))

In [106]:
test_doc_vecs = []
for doc in X_test:    
    doc_words = [term for term in doc if term not in stop_words]
    test_doc_vecs.append(sent_vec_w2v(doc_words))

#### Logistic regression

In [107]:
from sklearn.linear_model import LogisticRegression

In [108]:
logreg = LogisticRegression(penalty="l1", random_state=42, C = 8)

In [109]:
logreg.fit(train_doc_vecs,y)

C:\Users\Naresh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Naresh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=8, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [110]:
pred_train_ys = logreg.predict(train_doc_vecs)
pred_test_ys = logreg.predict(test_doc_vecs)
print("Train accuracy: ", accuracy_score(pred_train_ys, y))
print("Test accuracy: ", accuracy_score(pred_test_ys, y_test))

Train accuracy:  0.9717411121239745
Test accuracy:  0.9639104613978986
